# 6.3语言模型数据集（周杰伦歌词）

## 6.3.1读取数据集

In [4]:
with open('./data/dive/jaychou_lyrics.txt',encoding="utf-8") as f:
    corpus_chars = f.read()
print(len(corpus_chars))
print(corpus_chars[: 40])

corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ') # 换行和回车换成空格
corpus_chars = corpus_chars[: 10000] # 只保留前10000个字符来训练模型

63282
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


## 6.3.2建立字符索引

In [5]:
# 将数据集中所有不同字符取出来，将其逐一映射到索引来构造词典
idx_to_char = list(set(corpus_chars)) # set去重，list得到索引到字符的映射
char_to_idx = {char: i for i, char in enumerate(idx_to_char)} # 字符到索引的映射
vocab_size = len(char_to_idx)
print(vocab_size)

1027


In [6]:
# 将每个字符转化为索引，得到一个索引的序列
corpus_indices = [char_to_idx[char] for char in corpus_chars]  
sample = corpus_indices[: 20]
# 索引先转化成对应字符，得到字符列表
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [841, 684, 736, 632, 579, 693, 737, 841, 684, 138, 856, 983, 747, 545, 905, 250, 737, 841, 684, 138]


In [7]:
def load_data_jay_lyrics():
    with open('./data/dive/jaychou_lyrics.txt',encoding="utf-8") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

## 6.3.3时序数据的采样

### 6.3.3.1随机采样

In [8]:
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    '''
    随机采样
    @corpus_indices：字符索引序列
    @batch_size：每个小批量的样本数
    @num_steps：每个样本所包含的时间步数
    '''
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次读取batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices] # 取出对应样本
        Y = [_data(j + 1) for j in batch_indices] # 取出对应标签
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [9]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18]]) 

X:  tensor([[18, 19, 20, 21, 22, 23],
        [ 0,  1,  2,  3,  4,  5]]) 
Y: tensor([[19, 20, 21, 22, 23, 24],
        [ 1,  2,  3,  4,  5,  6]]) 



### 6.3.3.2相邻采样

In [10]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    '''
    相邻采样
    '''
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 保证corpus_indices的长度可以整除批量大小
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps # 样本不能包含最后的字符
    for i in range(batch_num):
        i = i * num_steps # 当前批量首字符对应的下标（列号）
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [11]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 

